## Exporting Model to TorchScript for Inference at Mobile Runtime

### Dependencies & Model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

In [2]:
sys.path.insert(0, '../src/pred/')
sys.path.insert(1, '../src/datasets')

from model import gazetrack_model
from gazetrack_data import gazetrack_dataset

In [4]:
model = gazetrack_model()
if(torch.cuda.is_available()):
    dev = torch.device('cuda:0')
else:
    dev = torch.device('cpu')

print(dev)
    
weights = torch.load("../checkpoints/GoogleCheckpoint_GoogleSplit.ckpt", map_location=dev)['state_dict']
model.load_state_dict(weights)
model.to(dev)
model.eval()
print(model)

cpu


RuntimeError: Error(s) in loading state_dict for gazetrack_model:
	size mismatch for lmModel.model.0.weight: copying a param with shape torch.Size([128, 8]) from checkpoint, the shape in current model is torch.Size([128, 11]).

## Fuse Layers
https://pytorch.org/docs/stable/generated/torch.quantization.fuse_modules.html

conv, bn
conv, bn, relu
conv, relu
linear, relu
bn, relu

Don't know if fusing is achievable here 

In [5]:
modules_to_fuse = []
modules_to_fuse.append(['conv_bn_relu.conv', 'conv_bn_relu.bn'])
torch.quantization.fuse_modules(model, modules_to_fuse, inplace=True)
print(model)

AttributeError: 'gazetrack_model' object has no attribute 'conv_bn_relu'

## Quantize

Static post-training quantization

In [19]:
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')

torch.quantization.prepare(model, inplace=True)

# use data set to calibrate quantized weights
def calibrate(model, calibration_data): 
    return

calibrate(model, [])

model = torch.quantization.convert(model, inplace=True)

/Users/jason/opt/anaconda3/envs/gaze-track/lib/python3.10/site-packages/torch/ao/quantization/observer.py:1135: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


## Save TorchScript Model for Inference at Mobile Runtime

In [21]:
from torch.utils.mobile_optimizer import optimize_for_mobile

torchscript_model = torch.jit.script(model)

torchscript_model_optimized = optimize_for_mobile(torchscript_model)

torch.jit.save(torchscript_model_optimized, 'model.pt')